API docs for the WikiSRATMicroService
===

**SRAT code is all in SQL, run from this notebook.**  
This code is within the GitHub repository. The result of this will be a table of NHDplus catchments. 

**Instances where load reduction is greater than the load created by an NHDplus catchment:**
- After weekly check in call, decided that it can be negative, so leave as raw value
- Otherwise can set threshold to where can't reduce loads beyond a proportion of the current load

In [1]:
import requests
import numpy as np
import pandas as pd
import geopandas as gpd 
from requests.auth import HTTPBasicAuth
import json
import os
import psycopg2
from sqlalchemy import create_engine 
from pathlib import Path
import pyarrow
from StringParser import StringParser
from DatabaseAdapter import DatabaseAdapter
from DatabaseFormatter import DatabaseFormatter
from DatabaseMakeTable import DatabaseMakeTable

# Run WikiSRATMicroService API

## Open Connection to WikiSRATMicroService Database

To fetch information to assist in formating requests to the WikiSRATMicroService API.

In [2]:
# GET THE DATABASE CONFIG INFORMATION USING A CONFIG FILE. 
# THE FILE IS IN THE GITIGNORE SO WILL REQUIRE BEING SENT VIA EMAIL.

config_file = json.load(open('db_config.json'))
PG_CONFIG = config_file['PG_CONFIG']

_host = PG_CONFIG['host'],
_database = PG_CONFIG['database'],
_user = PG_CONFIG['user'],
_password = PG_CONFIG['password'],
_port = PG_CONFIG['port']

### Create Connection to database with `psycopg2`, to read and write

In [3]:
# Create connection to database using `psycopg2`

_PG_Connection = psycopg2.connect(
        host=PG_CONFIG['host'],
        database=PG_CONFIG['database'],
        user=PG_CONFIG['user'],
        password=PG_CONFIG['password'],
        port=PG_CONFIG['port'])

### Create Connection using SQLAlchemy, to read with GeoPandas
An alternate way to connect, using sqlalchemy, written by Sarah.  
Benefits include:
- Doesn't require knowledge of column names
- Correctly auto-parses datatypes
- Auto-converts column named 'geom' from a string into a geometry datatype compatible with  GeoPandas and most plotting libraries.

In [4]:
%%time
# Connect to database with sqlalchemy

db_connection_url = "postgresql://{}:{}@{}:{}/{}".format(_user[0], _password[0], _host[0], _port, _database[0])
con = create_engine(db_connection_url)  

Wall time: 78.5 ms


## Fetch data for formating requests to the WikiSRATMicroService API


In [5]:
%%time

# GET THE MODELED LOADS FROM THE DRWI DATABASE, DERIVED FROM MMW MODEL RUNS

_PG_Connection.set_isolation_level(0)
_cur = _PG_Connection.cursor()
_cur.execute("select * from databmpapi.drb_loads_raw order by huc12;")  
# _cur.execute("select * from databmpapi.drb_loads_raw where huc12 in ('020402030902', '020402030901');")  

_dbdata = _cur.fetchall()
print(len(_dbdata))

484
Wall time: 862 ms


## Run WikiSRATMicroService API


In [6]:
# CREATE A COUPLE HELPER FUNCTIONS TO RUN THE MICROSERVICE
def respond(err, res=None):
    return {
        'statusCode': '400' if err else '200',
        'body': err.args[0] if err else json.dumps(res),
        'headers': {
            'Content-Type': 'application/json',
            'Access-Control-Allow-Origin': '*'
        },
    }

def lambda_handler(event, context):
    try:
        data = StringParser.parse(event['body'])
        db = DatabaseAdapter(_database[0], _user[0], _host[0], _port, _password[0], _flag)
        input_array = DatabaseAdapter.python_to_array(data)
        return respond(None, db.run_model(input_array))
    except AttributeError as e:
        return respond(e)

In [7]:
# Prepare the input payload body for the MicroService request
_body = DatabaseFormatter.parse(_dbdata)
# _body = '[{"huc12": "020402010101", "tpload_hp": 10, "tpload_crop": 10, "tpload_wooded": 10, "tpload_open": 10, "tpload_barren": 10, "tpload_ldm": 10, "tpload_mdm": 10, "tpload_hdm": 10, "tpload_wetland": 10, "tpload_farman": 10, "tpload_tiledrain": 10, "tpload_streambank": 10, "tpload_subsurface": 10, "tpload_pointsource": 10, "tpload_septics": 10, "tnload_hp": 10, "tnload_crop": 10, "tnload_wooded": 10, "tnload_open": 10, "tnload_barren": 10, "tnload_ldm": 10, "tnload_mdm": 10, "tnload_hdm": 10, "tnload_wetland": 10, "tnload_farman": 10, "tnload_tiledrain": 10, "tnload_streambank": 10, "tnload_subsurface": 10, "tnload_pointsource": 10, "tnload_septics": 10, "tssload_hp": 10, "tssload_crop": 10, "tssload_wooded": 10, "tssload_open": 10, "tssload_barren": 10, "tssload_ldm": 10, "tssload_mdm": 10, "tssload_hdm": 10, "tssload_wetland": 10, "tssload_tiledrain": 10, "tssload_streambank": 10}, {"huc12": "020402010102", "tpload_hp": 10, "tpload_crop": 10, "tpload_wooded": 10, "tpload_open": 10, "tpload_barren": 10, "tpload_ldm": 10, "tpload_mdm": 10, "tpload_hdm": 10, "tpload_wetland": 10, "tpload_farman": 10, "tpload_tiledrain": 10, "tpload_streambank": 10, "tpload_subsurface": 10, "tpload_pointsource": 10, "tpload_septics": 10, "tnload_hp": 10, "tnload_crop": 10, "tnload_wooded": 10, "tnload_open": 10, "tnload_barren": 10, "tnload_ldm": 10, "tnload_mdm": 10, "tnload_hdm": 10, "tnload_wetland": 10, "tnload_farman": 10, "tnload_tiledrain": 10, "tnload_streambank": 10, "tnload_subsurface": 10, "tnload_pointsource": 10, "tnload_septics": 10, "tssload_hp": 10, "tssload_crop": 10, "tssload_wooded": 10, "tssload_open": 10, "tssload_barren": 10, "tssload_ldm": 10, "tssload_mdm": 10, "tssload_hdm": 10, "tssload_wetland": 10, "tssload_tiledrain": 10, "tssload_streambank": 10}, {"huc12": "020402010103", "tpload_hp": 10, "tpload_crop": 10, "tpload_wooded": 10, "tpload_open": 10, "tpload_barren": 10, "tpload_ldm": 10, "tpload_mdm": 10, "tpload_hdm": 10, "tpload_wetland": 10, "tpload_farman": 10, "tpload_tiledrain": 10, "tpload_streambank": 10, "tpload_subsurface": 10, "tpload_pointsource": 10, "tpload_septics": 10, "tnload_hp": 10, "tnload_crop": 10, "tnload_wooded": 10, "tnload_open": 10, "tnload_barren": 10, "tnload_ldm": 10, "tnload_mdm": 10, "tnload_hdm": 10, "tnload_wetland": 10, "tnload_farman": 10, "tnload_tiledrain": 10, "tnload_streambank": 10, "tnload_subsurface": 10, "tnload_pointsource": 10, "tnload_septics": 10, "tssload_hp": 10, "tssload_crop": 10, "tssload_wooded": 10, "tssload_open": 10, "tssload_barren": 10, "tssload_ldm": 10, "tssload_mdm": 10, "tssload_hdm": 10, "tssload_wetland": 10, "tssload_tiledrain": 10, "tssload_streambank": 10}]'

In [8]:
_body?

Type:        str
String form: [{"huc12": "020401010101", "tpload_hp": 607.6, "tpload_crop": 362.6, "tpload_wooded": 36.6, "tplo <...> hdm": 2799.2, "tssload_wetland": 65.3, "tssload_tiledrain": 0.0, "tssload_streambank": 780437.0}]
Length:      483251
Docstring:  
str(object='') -> str
str(bytes_or_buffer[, encoding[, errors]]) -> str

Create a new string object from the given object. If encoding or
errors is specified, then the object must expose a data buffer
that will be decoded using the given encoding and error handler.
Otherwise, returns the result of object.__str__() (if defined)
or repr(object).
encoding defaults to sys.getdefaultencoding().
errors defaults to 'strict'.


### for Base Model

In [9]:
%%time

# FOR ALL DRWI HUC12s, FEED THROUGH THE MICROSERVICE TO GET SUB-BASIN ATTENUATION
# The database adapter routine flag can either be 'base' or 'restoration', depending on if you want these
# projects to be removed from the attenuation routine. Restoration projects come from what was enetered in
# through FieldDoc.

_flag = 'base'

# RUN THE HUC12s THROUGH THE MICROSERVICE
_r = dict(lambda_handler({"body": _body},None))

Wall time: 6.69 s


In [10]:
# Explore the API response
_r?

Type:        dict
String form: {'statusCode': '200', 'body': '{"huc12s": {"020401010101": {"huc12": "020401010101", "tpload_hp": <...> 15247}}}}}', 'headers': {'Content-Type': 'application/json', 'Access-Control-Allow-Origin': '*'}}
Length:      3
Docstring:  
dict() -> new empty dictionary
dict(mapping) -> new dictionary initialized from a mapping object's
    (key, value) pairs
dict(iterable) -> new dictionary initialized as if via:
    d = {}
    for k, v in iterable:
        d[k] = v
dict(**kwargs) -> new dictionary initialized with the name=value pairs
    in the keyword argument list.  For example:  dict(one=1, two=2)


In [11]:
# Extract the NHD Loads from the response
_nhdloads = dict(json.loads(_r['body']))['huc12s']
_nhdloads?

Type:        dict
String form: {'020401010101': {'huc12': '020401010101', 'tpload_hp': 602.07665754054, 'tpload_crop': 360.72733 <...> 0436867554431356, 'tssloadrate_total': 25055.0654476434, 'tssloadrate_conc': 41.5994419315247}}}}
Length:      484
Docstring:  
dict() -> new empty dictionary
dict(mapping) -> new dictionary initialized from a mapping object's
    (key, value) pairs
dict(iterable) -> new dictionary initialized as if via:
    d = {}
    for k, v in iterable:
        d[k] = v
dict(**kwargs) -> new dictionary initialized with the name=value pairs
    in the keyword argument list.  For example:  dict(one=1, two=2)


In [12]:
# Explore selection of data for a HUC12
print(dict(json.loads(_r['body']))['huc12s']['020402010101']['catchments'])

{'4481881': {'comid': 4481881, 'tploadrate_total': 8.67678906666698, 'tploadate_conc': 0.00480867068316327, 'tnloadrate_total': 205.478173246534, 'tnloadrate_conc': 0.188308808608071, 'tssloadrate_total': 12430.9823731758, 'tssloadrate_conc': 10.6329585165398}, '4481681': {'comid': 4481681, 'tploadrate_total': 45.9269178229512, 'tploadate_conc': 0.0139356826495012, 'tnloadrate_total': 1098.12798966065, 'tnloadrate_conc': 0.333206840298741, 'tssloadrate_total': 128512.512354698, 'tssloadrate_conc': 38.9947698116636}, '4481279': {'comid': 4481279, 'tploadrate_total': 9.3369485692889, 'tploadate_conc': 0.0870941956538323, 'tnloadrate_total': 260.499630512306, 'tnloadrate_conc': 2.34191601922763, 'tssloadrate_total': 4135.72100063273, 'tssloadrate_conc': 49.6417992751781}, '4481935': {'comid': 4481935, 'tploadrate_total': 47.8207522003034, 'tploadate_conc': 0.107891339771179, 'tnloadrate_total': 1122.82652578667, 'tnloadrate_conc': 2.53327797292518, 'tssloadrate_total': 29233.9090668937, '

# Load Results into Database (SKIP if DONE)

### If you have done this for the latest data, skip this block.

Loading results into the database is a time consuming process (~20-30 minutes), so it only needs to be run once for every data update. 

In [13]:
# GET THE TOTAL NUMBER OF ROWS TO PRINT THE % COMPLETED LATER ON

t = 0
for huc12s, huc12 in _nhdloads.items():
    for comid in _nhdloads[huc12s]['catchments']:
        t += 1
        
t

19496

## Save "Base" Model Results (i.e. no BMPs) to Database

In [14]:
# LOAD THE RESULTS INTO A DATABASE FOR REVIEW, CONSULT MSC94@DREXEL.EDU FOR MORE INFORMATION (MAY REQUIRE PERMISSION)
# CREATE THE TABLE TO CACHE THE API OUTPUT
# This uses an imported function to create the table. This is necessary to get the COMID geometries

# SET THE TABLE NAME AND CREATE TABLE
tablename_base = 'base_run'
new = DatabaseMakeTable(_database[0], _user[0], _host[0], _port, _password[0], tablename_base)
new.make_table()


Table Created


In [15]:
%%time

# LOADING RESULTS INTO THE DB CAN TAKE 10-15 MINUTES with a fast internet connection 
# (~ 10 min for Mike; ~14 min for Anthony at work),
# and it times-out after only 20% complete with a slower connection (i.e. Anthony's home)

c = 0
prog_update = 0.1
print('0%', end='--->')
for huc12s, huc12 in _nhdloads.items():
    for comid in _nhdloads[huc12s]['catchments']:
        update_arr = [int(_nhdloads[huc12s]['catchments'][comid]['comid']),
                      _nhdloads[huc12s]['catchments'][comid]['tploadrate_total'],
                      _nhdloads[huc12s]['catchments'][comid]['tploadate_conc'],
                      _nhdloads[huc12s]['catchments'][comid]['tnloadrate_total'],
                      _nhdloads[huc12s]['catchments'][comid]['tnloadrate_conc'],
                      _nhdloads[huc12s]['catchments'][comid]['tssloadrate_total'],
                      _nhdloads[huc12s]['catchments'][comid]['tssloadrate_conc']]
        update_arr = [x if x != None else -9999 for x in update_arr]
        _PG_Connection.set_isolation_level(0)
        _cur = _PG_Connection.cursor()
        _cur.execute("insert into wikiwtershedoutputs.{} values ({},{},{},{},{},{},{})"
                     ";".format(tablename_base, update_arr[0],update_arr[1],update_arr[2],update_arr[3],update_arr[4],update_arr[5],update_arr[6]))
        c += 1
        if c == int(t * prog_update - 1):
            print('{}%'.format(int(prog_update*100)), end='--->')
            prog_update = round(prog_update+0.1,1)
print('done')

0%--->10%--->20%--->30%--->40%--->50%--->60%--->70%--->80%--->90%--->100%--->done
CPU times: user 1.91 s, sys: 989 ms, total: 2.9 s
Wall time: 14min 46s


## Save "Restoration" Model Results (i.e. with BMPs) to Database

### Run WikiSRATMicroService API for Restoration

Reruns the WikiSRATMicroService API with restoration BMPs.

In [14]:
# FOR ALL DRWI HUC12s, FEED THROUGH THE MICROSERVICE TO GET SUB-BASIN ATTENUATION

# NOW RUN THE ATTENUATION WITH THE BMPs ADDED IN

_flag = 'restoration'

_r = dict(lambda_handler({"body": _body},None))
print('done')

done


In [15]:
# GET THE TOTAL NUMBER OF ROWS TO PRINT THE % COMPLETED LATER ON
_nhdloads = dict(json.loads(_r['body']))['huc12s']
t = 0
for huc12s, huc12 in _nhdloads.items():
    for comid in _nhdloads[huc12s]['catchments']:
        t += 1

### Save Restoration model to database

In [16]:
# LOAD THE RESULTS INTO A DATABASE FOR REVIEW, CONSULT MSC94@DREXEL.EDU FOR MORE INFORMATION (MAY REQUIRE PERMISSION)
# CREATE THE TABLE TO CACHE THE API OUTPUT
tablename_rest = 'restoration_run'
new = DatabaseMakeTable(_database[0], _user[0], _host[0], _port, _password[0], tablename_rest)
new.make_table()

Table Created


In [17]:
%%time

# LOADING RESULTS INTO THE DB CAN TAKE 10-15 MINUTES with a fast internet connection 
# (~ 10 min for Mike; ~14 min for Anthony at work),
# and it times-out after only 20% complete with a slower connection (i.e. Anthony's home)

c = 0
prog_update = 0.1
print('0%', end='--->')
for huc12s, huc12 in _nhdloads.items():
    for comid in _nhdloads[huc12s]['catchments']:
        update_arr = [int(_nhdloads[huc12s]['catchments'][comid]['comid']),
                      _nhdloads[huc12s]['catchments'][comid]['tploadrate_total'],
                      _nhdloads[huc12s]['catchments'][comid]['tploadate_conc'],
                      _nhdloads[huc12s]['catchments'][comid]['tnloadrate_total'],
                      _nhdloads[huc12s]['catchments'][comid]['tnloadrate_conc'],
                      _nhdloads[huc12s]['catchments'][comid]['tssloadrate_total'],
                      _nhdloads[huc12s]['catchments'][comid]['tssloadrate_conc']]
        update_arr = [x if x != None else -9999 for x in update_arr]
        _PG_Connection.set_isolation_level(0)
        _cur = _PG_Connection.cursor()
        _cur.execute("insert into wikiwtershedoutputs.{} values ({},{},{},{},{},{},{})"
                     ";".format(tablename_rest, update_arr[0],update_arr[1],update_arr[2],update_arr[3],update_arr[4],update_arr[5],update_arr[6]))
        c += 1
        if c == int(t * prog_update - 1):
            print('{}%'.format(int(prog_update*100)), end='--->')
            prog_update = round(prog_update+0.1,1)

print('done')

0%--->10%--->20%--->30%--->40%--->50%--->60%--->70%--->80%--->90%--->100%--->done
CPU times: user 2.42 s, sys: 1.33 s, total: 3.75 s
Wall time: 14min 4s


# ABOVE CODE (Load Results) ONLY NEEDS TO BE RUN ONCE

Use labs to get outline to collapse code to be run once. Further edits to the data frames for API output can be done here.

# Read Results to Pandas and GeoPandas

In [20]:
# PUT HERE ANY TABLE NAMES CREATED ABOVE, IN CASE THE KERNEL WAS RESTARTED AND YOU DON'T WANT TO RUN THE ABOVE AGAIN
tablename_rest = 'restoration_run'
tablename_base = 'base_run'
tablename_cluster = 'clustersv'

## Read from Database into Pandas with `psycopg2`
Using a custom function written by Mike, using the [pyscopg2](https://www.psycopg.org) library.

In [5]:
# Re-connect to database using `psycopg2`(necessary if timed out)

_PG_Connection = psycopg2.connect(
        host=PG_CONFIG['host'],
        database=PG_CONFIG['database'],
        user=PG_CONFIG['user'],
        password=PG_CONFIG['password'],
        port=PG_CONFIG['port'])

## Read MicroService Outputs from Database using SQLAlchemy and GeoPandas
An alternate way to connect, using sqlalchemy, written by Sarah.  
Benefits include:
- Doesn't require knowledge of column names
- Correctly auto-parses datatypes
- Auto-converts column named 'geom' from a string into a geometry datatype compatible with  GeoPandas and most plotting libraries.

From Mike:
- Loadrate_total_ws was my attempt to get the loadrate totals to kg per hectare
  - Fails if mean annual flow (maflowv (CFS)) doesn’t exist (-9999).
  - loadrate_total_ws  = ((loadate_conc * 28.3168 * 31557600 / 1000000) * maflowv) / watershed_hectares
    - 31557600 = 365.25*24*60*60
    - 28.3168 liters in a cubic foot
    - 1000000 mg in kg

In [23]:
%%time
# Connect to database with sqlalchemy
from sqlalchemy import create_engine  

db_connection_url = "postgresql://{}:{}@{}:{}/{}".format(_user[0], _password[0], _host[0], _port, _database[0])
con = create_engine(db_connection_url)  

Wall time: 0 ns


In [24]:
%%time
# import data with geopandas compatible geometry
import geopandas as gpd 

tablename_rest = 'restoration_run'
tablename_base = 'base_run'

base_model_select = 'SELECT * FROM wikiwtershedoutputs.{}'.format(tablename_base)
rest_model_select = 'SELECT * FROM wikiwtershedoutputs.{}'.format(tablename_rest)

base_df = gpd.read_postgis(base_model_select, con)
rest_df = gpd.read_postgis(rest_model_select, con)

Wall time: 54.1 s


In [25]:
# read phase information
phase_select = 'SELECT * FROM datapolassess.focusarea_comids'
phase_df = pd.read_sql(phase_select, con)

In [26]:
base_df.head(3)

,comid,tploadrate_total,tploadate_conc,tnloadrate_total,tnloadate_conc,tssloadrate_total,tssloadate_conc,catchment_hectares,watershed_hectares,tploadrate_total_ws,...,geom,geom_catchment,cluster,fa_name,sub_focusarea,nord,nordstop,huc12,streamorder,headwater
0,2743190,1.1681,0.0406,8.2907,0.3244,3109.3056,25.9329,1.3489,52912.98,0.2434,...,MULTILINESTRING Z ((484690.941 4591940.664 0.0...,01060000206A7F00000100000001030000000100000011...,drb,None,NaN,73007.0,73238.0,020401030601,4,0
1,2743192,0.0372,0.0374,0.8810,0.3108,19.4252,23.0368,0.1799,21557.61,0.2299,...,MULTILINESTRING Z ((484749.246 4591859.297 0.0...,01060000206A7F00000100000001030000000100000006...,drb,None,NaN,72879.0,73006.0,020401030601,4,0
2,2742506,0.0902,0.0244,0.9787,0.3380,196.7743,20.6736,2.2482,136291.50,0.1486,...,MULTILINESTRING Z ((487525.458 4591840.091 0.0...,01060000206A7F00000100000001030000000100000014...,drb,None,NaN,72529.0,73250.0,020401030603,6,0


In [27]:
rest_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 19496 entries, 0 to 19495
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   comid                 19496 non-null  int64   
 1   tploadrate_total      19496 non-null  float64 
 2   tploadate_conc        19496 non-null  float64 
 3   tnloadrate_total      19496 non-null  float64 
 4   tnloadate_conc        19496 non-null  float64 
 5   tssloadrate_total     19496 non-null  float64 
 6   tssloadate_conc       19496 non-null  float64 
 7   catchment_hectares    19496 non-null  float64 
 8   watershed_hectares    19496 non-null  float64 
 9   tploadrate_total_ws   19496 non-null  float64 
 10  tnloadrate_total_ws   19496 non-null  float64 
 11  tssloadrate_total_ws  19496 non-null  float64 
 12  maflowv               19496 non-null  float64 
 13  geom                  19494 non-null  geometry
 14  geom_catchment        19496 non-null  object  

### Set Index to COMID
For easier search and error-free merging.  
Doc: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.set_index.html

In [28]:
# Set index to COMID
base_df.set_index('comid', inplace=True)
rest_df.set_index('comid', inplace=True)

In [29]:
phase_df.set_index('comid', inplace=True)

### Add Focus Area Phase Information to Base and Rest DFs 

In [32]:
phase_df.columns

Index(['phase', 'fa_name', 'cluster', 'catchment'], dtype='object')

In [35]:
base_df.drop(['fa_name'], axis=1, inplace=True)
rest_df.drop(['fa_name'], axis=1, inplace=True)
phase_df.drop(['cluster', 'catchment'], axis=1, inplace=True)

In [39]:
base_df = base_df.join(phase_df)
rest_df = rest_df.join(phase_df)

### Sort Index
For performance.  
Docs: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_index.html

In [41]:
# Sort Index
base_df.sort_index(inplace=True)
rest_df.sort_index(inplace=True)

In [42]:
base_df.index

Int64Index([  1748535,   1748537,   1748539,   1748541,   1748543,   1748545,
              1748547,   1748549,   1748551,   1748553,
            ...
            932040361, 932040362, 932040363, 932040364, 932040365, 932040366,
            932040367, 932040368, 932040369, 932040370],
           dtype='int64', name='comid', length=19496)

In [43]:
rest_df.index

Int64Index([  1748535,   1748537,   1748539,   1748541,   1748543,   1748545,
              1748547,   1748549,   1748551,   1748553,
            ...
            932040361, 932040362, 932040363, 932040364, 932040365, 932040366,
            932040367, 932040368, 932040369, 932040370],
           dtype='int64', name='comid', length=19496)

### Confirm Differences between Base & Restoration

In [44]:
diff_df = base_df.tploadrate_total == rest_df.tploadrate_total
diff_df

comid
1748535      True
1748537      True
1748539      True
1748541      True
1748543      True
             ... 
932040366    True
932040367    True
932040368    True
932040369    True
932040370    True
Name: tploadrate_total, Length: 19496, dtype: bool

In [45]:
diff_df.value_counts()

True     19219
False      277
Name: tploadrate_total, dtype: int64

In [46]:
diff_df = base_df.tploadate_conc == rest_df.tploadate_conc
diff_df.value_counts()

True     18616
False      880
Name: tploadate_conc, dtype: int64

### Convert Nord, NordStop, Sub_FocusArea from Float to Nullable Integers
For improved performance, memory use, and compression.
Floats are generally problematic for any index or field used as an index.

Nord & NordStop contain null values, which by default Pandas interprets as a `numpy.nan` that is a `float64` dtype. This forces the entire series to `float64`.

Pandas can represent integer data with missing values (since v1.0) using `arrays.IntegerArray` and `dtype=pd.Int64Dtype()`. 

Docs:
- https://pandas.pydata.org/pandas-docs/stable/user_guide/integer_na.html
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.astype.html

In [47]:
base_df.sub_focusarea = base_df.sub_focusarea.astype(pd.Int64Dtype())
base_df.nord          = base_df.nord.astype(pd.Int64Dtype())
base_df.nordstop      = base_df.nordstop.astype(pd.Int64Dtype())

rest_df.sub_focusarea = rest_df.sub_focusarea.astype(pd.Int64Dtype())
rest_df.nord          = rest_df.nord.astype(pd.Int64Dtype())
rest_df.nordstop      = rest_df.nordstop.astype(pd.Int64Dtype()) 

In [48]:
base_df[base_df.sub_focusarea.notna()].head(3)

,tploadrate_total,tploadate_conc,tnloadrate_total,tnloadate_conc,tssloadrate_total,tssloadate_conc,catchment_hectares,watershed_hectares,tploadrate_total_ws,tnloadrate_total_ws,...,geom_catchment,cluster,sub_focusarea,nord,nordstop,huc12,streamorder,headwater,phase,fa_name
comid,,,,,,,,,,,,,,,,,,,,,
2583353,177.7780,0.0281,2200.1500,0.3300,104612.1133,25.3955,486.6020,1723.41,0.1870,2.1958,...,01060000206A7F0000010000000103000000010000002D...,New Jersey Highlands,3,70054,70057,020401050102,2,0,Phase 2,Paulinskill
2583369,3.8052,0.0802,38.6503,1.1833,10032.3886,82.1927,7.0144,1087.56,0.5157,7.6090,...,01060000206A7F00000100000001030000000100000018...,New Jersey Highlands,8,70152,70157,020401050102,2,0,Phase 2,Paulinskill
2583379,47.4438,0.0609,1163.0701,1.0621,40602.0194,68.6833,114.2091,18845.46,0.3768,6.5716,...,01060000206A7F000001000000010300000001000000BA...,New Jersey Highlands,1,70059,70157,020401050102,4,0,Phase 2,Paulinskill


### Convert String Objects to Categoricals
For improved performance, memory use, and compression.  
See https://anaconda.org/TomAugspurger/pandas-performance/notebook

Doc: https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html

In [34]:
%%time
# Compare before and after
base_df.huc12.count()

Wall time: 2 ms


19496

In [49]:
base_df.cluster = base_df.cluster.astype('category')
base_df.fa_name = base_df.fa_name.astype('category')
base_df.phase = base_df.phase.astype('category')
base_df.huc12   = base_df.huc12.astype('category')

rest_df.cluster = rest_df.cluster.astype('category')
rest_df.fa_name = rest_df.fa_name.astype('category')
rest_df.huc12   = rest_df.huc12.astype('category')
rest_df.phase = rest_df.phase.astype('category')

In [50]:
base_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 19496 entries, 1748535 to 932040370
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   tploadrate_total      19496 non-null  float64 
 1   tploadate_conc        19496 non-null  float64 
 2   tnloadrate_total      19496 non-null  float64 
 3   tnloadate_conc        19496 non-null  float64 
 4   tssloadrate_total     19496 non-null  float64 
 5   tssloadate_conc       19496 non-null  float64 
 6   catchment_hectares    19496 non-null  float64 
 7   watershed_hectares    19496 non-null  float64 
 8   tploadrate_total_ws   19496 non-null  float64 
 9   tnloadrate_total_ws   19496 non-null  float64 
 10  tssloadrate_total_ws  19496 non-null  float64 
 11  maflowv               19496 non-null  float64 
 12  geom                  19494 non-null  geometry
 13  geom_catchment        19496 non-null  object  
 14  cluster               17358 non-null

In [51]:
%%time
# This is now an order of magnitude faster
base_df.huc12.count()

Wall time: 0 ns


19496

In [52]:
%%time
base_df.huc12.unique()

Wall time: 11.4 ms


['020401020302', '020401020305', '020401020402', '020401020303', '020401020306', ..., '020402060604', '020402060504', '020402060605', '020402060103', '020402050701']
Length: 484
Categories (484, object): ['020401010101', '020401010102', '020401010103', '020401010104', ..., '020403020406', '020403020407', '020403020408', '020403030101']

In [53]:
%%time
base_df.huc12.value_counts()

Wall time: 9.66 ms


020403010704    310
020403010610    256
020403010406    253
020402060605    239
020402070203    231
               ... 
020402070605      2
020401060812      2
020403030101      2
020401020301      1
020401020401      1
Name: huc12, Length: 484, dtype: int64

### Correct Names

**Note on `ploadrate_total`, etc.**  
The microservice output says "tploadrate_total", however Mike suspected this is a mistake, and it actually is the local NHDplus **catchment annual load (kg/ha)**. We confirmed this by comparing to Model My Watershed subbasin modeling output for specific COMDIDs.
- The stream concentrations take into account the upstream watershed load, but this value is not returned. Mike calculated this when exporting the microservice results to PG for convenience, and saved as `tploadrate_total_ws` etc.

This code block assigns more accurate, shorter names to selected columns. All fields represent totals for the reach or catcment, so we are removing that from the name to shorten it.

Docs: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html

In [54]:
base_df.rename(columns={'tploadrate_total':'tp_load',
                        'tploadate_conc':'tp_conc',
                        'tnloadrate_total':'tn_load',
                        'tnloadate_conc':'tn_conc',
                        'tssloadrate_total':'tss_load',
                        'tssloadate_conc':'tss_conc',
                        'tploadrate_total_ws':'tp_loadrate_ws',
                        'tnloadrate_total_ws':'tn_loadrate_ws',
                        'tssloadrate_total_ws':'tss_loadrate_ws',
                       },
               inplace=True,
              )
base_df.head(3)

,tp_load,tp_conc,tn_load,tn_conc,tss_load,tss_conc,catchment_hectares,watershed_hectares,tp_loadrate_ws,tn_loadrate_ws,...,geom_catchment,cluster,sub_focusarea,nord,nordstop,huc12,streamorder,headwater,phase,fa_name
comid,,,,,,,,,,,,,,,,,,,,,
1748535,881.4618,0.0226,10322.4299,0.2643,390433.1419,9.9983,6496.7052,6501.69,0.1357,1.5874,...,01060000206A7F00000100000001030000000100000021...,drb,<NA>,74914,74914,020401020302,1,1,NaN,NaN
1748537,296.6355,0.0297,3165.6081,0.3166,88090.7401,8.8103,1663.1712,1664.46,0.1784,1.9019,...,01060000206A7F000001000000010300000001000000FC...,drb,<NA>,74913,74913,020401020302,1,1,NaN,NaN
1748539,350.9217,0.0350,2816.4257,0.2808,117212.5160,11.6874,1639.4128,1640.70,0.2139,1.7164,...,01060000206A7F000001000000010300000001000000C2...,drb,<NA>,74921,74921,020401020305,1,1,NaN,NaN


In [55]:
rest_df.rename(columns={'tploadrate_total':'tp_load',
                        'tploadate_conc':'tp_conc',
                        'tnloadrate_total':'tn_load',
                        'tnloadate_conc':'tn_conc',
                        'tssloadrate_total':'tss_load',
                        'tssloadate_conc':'tss_conc',
                        'tploadrate_total_ws':'tp_loadrate_ws',
                        'tnloadrate_total_ws':'tn_loadrate_ws',
                        'tssloadrate_total_ws':'tss_loadrate_ws',
                       },
               inplace=True,
              )
rest_df.head(3)

,tp_load,tp_conc,tn_load,tn_conc,tss_load,tss_conc,catchment_hectares,watershed_hectares,tp_loadrate_ws,tn_loadrate_ws,...,geom_catchment,cluster,sub_focusarea,nord,nordstop,huc12,streamorder,headwater,phase,fa_name
comid,,,,,,,,,,,,,,,,,,,,,
1748535,881.4618,0.0226,10322.4299,0.2643,390433.1419,9.9983,6496.7052,6501.69,0.1357,1.5874,...,01060000206A7F00000100000001030000000100000021...,drb,<NA>,74914,74914,020401020302,1,1,NaN,NaN
1748537,296.6355,0.0297,3165.6081,0.3166,88090.7401,8.8103,1663.1712,1664.46,0.1784,1.9019,...,01060000206A7F000001000000010300000001000000FC...,drb,<NA>,74913,74913,020401020302,1,1,NaN,NaN
1748539,350.9217,0.0350,2816.4257,0.2808,117212.5160,11.6874,1639.4128,1640.70,0.2139,1.7164,...,01060000206A7F000001000000010300000001000000C2...,drb,<NA>,74921,74921,020401020305,1,1,NaN,NaN


### Add Units (Not Implemented)

Loads are in kg/y.  
Concentrations are in mg/L.

From Mike:
- Loadrate_total_ws was my attempt to get the loadrate totals to kg per hectare
  - Fails if mean annual flow (maflowv (CFS)) doesn’t exist (-9999).
  - `loadrate_total_ws  = ((loadate_conc * 28.3168 * 31557600 / 1000000) * maflowv) / watershed_hectares`
    - `31557600 = 365.25 * 24 * 60 * 60`
    - 28.3168 liters in a cubic foot
    - 1000000 mg in kg

From: https://www.fws.gov/r5gomp/gom/nhd-gom/NHDPLUS_UserGuide.pdf
> MAFlowV: Mean Annual Flow (cfs) at bottom of flowline as computed by Vogel Method

**INCOMPLETE**

Options for the future:
- https://stackoverflow.com/questions/14688306/adding-meta-information-metadata-to-pandas-dataframe
- https://pandas.pydata.org/docs/reference/api/pandas.Series.attrs.html
- https://stackoverflow.com/questions/39419178/how-can-i-manage-units-in-pandas-data
- https://github.com/hgrecco/pint
- https://towardsdatascience.com/add-metadata-to-your-dataset-using-apache-parquet-75360d2073bd

In [56]:
#base_df.attrs

In [57]:
#base_df.tp_load.attrs

## Convert to Reach-Specific GeoDataFrame

A GeoPandas DataFrame can only have one geometry column, and the auto-import defaulted to the reach polyline. So create a new DataFrame and drop catchment-specific columns.

In [58]:
base_df_reach = base_df.drop(['tp_load',
                              'tn_load',
                              'tss_load',
                              'tp_loadrate_ws',
                              'tn_loadrate_ws',
                              'tss_loadrate_ws',
                              'geom_catchment',
                             ], axis=1)
rest_df_reach = rest_df.drop(['tp_load',
                              'tn_load',
                              'tss_load',
                              'tp_loadrate_ws',
                              'tn_loadrate_ws',
                              'tss_loadrate_ws',
                              'geom_catchment',
                             ], axis=1)

In [59]:
base_df_reach.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 19496 entries, 1748535 to 932040370
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   tp_conc             19496 non-null  float64 
 1   tn_conc             19496 non-null  float64 
 2   tss_conc            19496 non-null  float64 
 3   catchment_hectares  19496 non-null  float64 
 4   watershed_hectares  19496 non-null  float64 
 5   maflowv             19496 non-null  float64 
 6   geom                19494 non-null  geometry
 7   cluster             17358 non-null  category
 8   sub_focusarea       186 non-null    Int64   
 9   nord                18870 non-null  Int64   
 10  nordstop            18844 non-null  Int64   
 11  huc12               19496 non-null  category
 12  streamorder         19496 non-null  int64   
 13  headwater           19496 non-null  int64   
 14  phase               4082 non-null   category
 15  fa_name           

In [60]:
rest_df_reach.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 19496 entries, 1748535 to 932040370
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   tp_conc             19496 non-null  float64 
 1   tn_conc             19496 non-null  float64 
 2   tss_conc            19496 non-null  float64 
 3   catchment_hectares  19496 non-null  float64 
 4   watershed_hectares  19496 non-null  float64 
 5   maflowv             19496 non-null  float64 
 6   geom                19494 non-null  geometry
 7   cluster             17358 non-null  category
 8   sub_focusarea       186 non-null    Int64   
 9   nord                18870 non-null  Int64   
 10  nordstop            18844 non-null  Int64   
 11  huc12               19496 non-null  category
 12  streamorder         19496 non-null  int64   
 13  headwater           19496 non-null  int64   
 14  phase               4082 non-null   category
 15  fa_name           

In [61]:
base_df_reach

,tp_conc,tn_conc,tss_conc,catchment_hectares,watershed_hectares,maflowv,geom,cluster,sub_focusarea,nord,nordstop,huc12,streamorder,headwater,phase,fa_name
comid,,,,,,,,,,,,,,,,
1748535,0.0226,0.2643,9.9983,6496.7052,6501.69,43.699,MULTILINESTRING Z ((539681.467 4689416.166 0.0...,drb,<NA>,74914,74914,020401020302,1,1,NaN,NaN
1748537,0.0297,0.3166,8.8103,1663.1712,1664.46,11.189,MULTILINESTRING Z ((532825.185 4684387.302 0.0...,drb,<NA>,74913,74913,020401020302,1,1,NaN,NaN
1748539,0.0350,0.2808,11.6874,1639.4128,1640.70,11.223,MULTILINESTRING Z ((524096.535 4677200.269 0.0...,drb,<NA>,74921,74921,020401020305,1,1,NaN,NaN
1748541,0.0245,0.2703,10.2217,3013.8348,12912.30,86.528,MULTILINESTRING Z ((533110.692 4677277.923 0.0...,drb,<NA>,74911,74915,020401020302,2,0,NaN,NaN
1748543,0.0307,0.2661,10.8301,1151.0990,5232.87,35.389,MULTILINESTRING Z ((526672.244 4673109.451 0.0...,drb,<NA>,74920,74922,020401020305,2,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
932040366,-9999.0000,-9999.0000,-9999.0000,2124.7248,2720941.47,17802.923,MULTILINESTRING Z ((460161.862 4401869.478 0.0...,drb,<NA>,65070,76964,020402060103,7,0,NaN,NaN
932040367,-9999.0000,-9999.0000,-9999.0000,788.7859,2717821.26,17788.281,MULTILINESTRING Z ((461055.049 4402946.027 0.0...,drb,<NA>,65079,76964,020402060103,7,0,NaN,NaN
932040368,-9999.0000,-9999.0000,-9999.0000,265.0275,2716120.08,17780.448,MULTILINESTRING Z ((462015.751 4404014.788 0.0...,drb,<NA>,65080,76960,020402060103,7,0,NaN,NaN


### Confirm Differences between Base & Restoration

In [62]:
diff_df = base_df_reach.tp_conc == rest_df_reach.tp_conc
diff_df.value_counts()

True     18616
False      880
Name: tp_conc, dtype: int64

### Replace -9999.0 Concentrations with NaN

**Question:** should we set these to zero rather than NaN?

In [63]:
base_df_reach.tp_conc[base_df_reach.tp_conc==-9999.0].count()

2673

In [64]:
rest_df_reach.tp_conc[rest_df_reach.tp_conc==-9999.0].count()

2673

In [65]:
import numpy as np
base_df_reach.replace(to_replace=-9999.0, value=np.NaN, inplace=True)
rest_df_reach.replace(to_replace=-9999.0, value=np.NaN, inplace=True)

In [66]:
base_df_reach.tp_conc[base_df_reach.tp_conc==-9999.0].count()

0

In [67]:
rest_df_reach.tp_conc[rest_df_reach.tp_conc==-9999.0].count()

0

In [68]:
rest_df_reach

,tp_conc,tn_conc,tss_conc,catchment_hectares,watershed_hectares,maflowv,geom,cluster,sub_focusarea,nord,nordstop,huc12,streamorder,headwater,phase,fa_name
comid,,,,,,,,,,,,,,,,
1748535,0.0226,0.2643,9.9983,6496.7052,6501.69,43.699,MULTILINESTRING Z ((539681.467 4689416.166 0.0...,drb,<NA>,74914,74914,020401020302,1,1,NaN,NaN
1748537,0.0297,0.3166,8.8103,1663.1712,1664.46,11.189,MULTILINESTRING Z ((532825.185 4684387.302 0.0...,drb,<NA>,74913,74913,020401020302,1,1,NaN,NaN
1748539,0.0350,0.2808,11.6874,1639.4128,1640.70,11.223,MULTILINESTRING Z ((524096.535 4677200.269 0.0...,drb,<NA>,74921,74921,020401020305,1,1,NaN,NaN
1748541,0.0245,0.2703,10.2217,3013.8348,12912.30,86.528,MULTILINESTRING Z ((533110.692 4677277.923 0.0...,drb,<NA>,74911,74915,020401020302,2,0,NaN,NaN
1748543,0.0307,0.2661,10.8301,1151.0990,5232.87,35.389,MULTILINESTRING Z ((526672.244 4673109.451 0.0...,drb,<NA>,74920,74922,020401020305,2,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
932040366,NaN,NaN,NaN,2124.7248,2720941.47,17802.923,MULTILINESTRING Z ((460161.862 4401869.478 0.0...,drb,<NA>,65070,76964,020402060103,7,0,NaN,NaN
932040367,NaN,NaN,NaN,788.7859,2717821.26,17788.281,MULTILINESTRING Z ((461055.049 4402946.027 0.0...,drb,<NA>,65079,76964,020402060103,7,0,NaN,NaN
932040368,NaN,NaN,NaN,265.0275,2716120.08,17780.448,MULTILINESTRING Z ((462015.751 4404014.788 0.0...,drb,<NA>,65080,76960,020402060103,7,0,NaN,NaN


In [69]:
diff_df = base_df_reach.tp_conc == rest_df_reach.tp_conc
diff_df.value_counts()

True     15943
False     3553
Name: tp_conc, dtype: int64

**NOTE: NaN comparisons produce FALSE**

## Convert to Catchment-Specific GeoDataFrame

A GeoPandas DataFrame can only have one geometry column, and the auto-import defaulted to the reach polyline. So create a new DataFrame and drop catchment-specific columns.

In [70]:
base_df_catch = base_df.drop(['tp_conc',
                              'tn_conc',
                              'tss_conc',
                              'geom',
                             ], axis=1)
rest_df_catch = rest_df.drop(['tp_conc',
                              'tn_conc',
                              'tss_conc',
                              'geom',
                             ], axis=1)

In [71]:
base_df_catch.head()

,tp_load,tn_load,tss_load,catchment_hectares,watershed_hectares,tp_loadrate_ws,tn_loadrate_ws,tss_loadrate_ws,maflowv,geom_catchment,cluster,sub_focusarea,nord,nordstop,huc12,streamorder,headwater,phase,fa_name
comid,,,,,,,,,,,,,,,,,,,
1748535,881.4618,10322.4299,390433.1419,6496.7052,6501.69,0.1357,1.5874,60.0509,43.699,01060000206A7F00000100000001030000000100000021...,drb,<NA>,74914,74914,020401020302,1,1,NaN,NaN
1748537,296.6355,3165.6081,88090.7401,1663.1712,1664.46,0.1784,1.9019,52.9245,11.189,01060000206A7F000001000000010300000001000000FC...,drb,<NA>,74913,74913,020401020302,1,1,NaN,NaN
1748539,350.9217,2816.4257,117212.5160,1639.4128,1640.70,0.2139,1.7164,71.4407,11.223,01060000206A7F000001000000010300000001000000C2...,drb,<NA>,74921,74921,020401020305,1,1,NaN,NaN
1748541,517.2223,5245.9416,201648.7618,3013.8348,12912.30,0.1467,1.6186,61.2103,86.528,01060000206A7F00000100000001030000000100000051...,drb,<NA>,74911,74915,020401020302,2,0,NaN,NaN
1748543,289.1315,2478.5764,108737.6061,1151.0990,5232.87,0.1855,1.6081,65.4499,35.389,01060000206A7F000001000000010300000001000000A1...,drb,<NA>,74920,74922,020401020305,2,0,NaN,NaN


In [72]:
rest_df_catch.head()

,tp_load,tn_load,tss_load,catchment_hectares,watershed_hectares,tp_loadrate_ws,tn_loadrate_ws,tss_loadrate_ws,maflowv,geom_catchment,cluster,sub_focusarea,nord,nordstop,huc12,streamorder,headwater,phase,fa_name
comid,,,,,,,,,,,,,,,,,,,
1748535,881.4618,10322.4299,390433.1419,6496.7052,6501.69,0.1357,1.5874,60.0509,43.699,01060000206A7F00000100000001030000000100000021...,drb,<NA>,74914,74914,020401020302,1,1,NaN,NaN
1748537,296.6355,3165.6081,88090.7401,1663.1712,1664.46,0.1784,1.9019,52.9245,11.189,01060000206A7F000001000000010300000001000000FC...,drb,<NA>,74913,74913,020401020302,1,1,NaN,NaN
1748539,350.9217,2816.4257,117212.5160,1639.4128,1640.70,0.2139,1.7164,71.4407,11.223,01060000206A7F000001000000010300000001000000C2...,drb,<NA>,74921,74921,020401020305,1,1,NaN,NaN
1748541,517.2223,5245.9416,201648.7618,3013.8348,12912.30,0.1467,1.6186,61.2103,86.528,01060000206A7F00000100000001030000000100000051...,drb,<NA>,74911,74915,020401020302,2,0,NaN,NaN
1748543,289.1315,2478.5764,108737.6061,1151.0990,5232.87,0.1855,1.6081,65.4499,35.389,01060000206A7F000001000000010300000001000000A1...,drb,<NA>,74920,74922,020401020305,2,0,NaN,NaN


### Confirm Differences between Base & Restoration

In [73]:
diff_df = base_df_catch.tp_load == rest_df_catch.tp_load
diff_df.value_counts()

True     19219
False      277
Name: tp_load, dtype: int64

### Read Database Again To Set Catchment Geometry
This reruns the `gpd.read_postgis()` function with the optional argument `geom_col="geom_catchment"`, as a workaround for converting the catchment boundaries to geometry datatypes.
Also only selects the two required fields to speed up the response.

In [74]:
%%time
# get catchment geometry
base_catch = 'SELECT comid, geom_catchment FROM wikiwtershedoutputs.{}'.format(tablename_base)
rest_catch = 'SELECT comid, geom_catchment FROM wikiwtershedoutputs.{}'.format(tablename_rest)

base_df_catch_geom = gpd.read_postgis(base_catch, con, geom_col="geom_catchment")
rest_df_catch_geom = gpd.read_postgis(rest_catch, con, geom_col="geom_catchment")

Wall time: 36 s


In [75]:
base_df_catch_geom.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 19496 entries, 0 to 19495
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   comid           19496 non-null  int64   
 1   geom_catchment  19496 non-null  geometry
dtypes: geometry(1), int64(1)
memory usage: 304.8 KB


In [76]:
base_df_catch_geom.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 19496 entries, 0 to 19495
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   comid           19496 non-null  int64   
 1   geom_catchment  19496 non-null  geometry
dtypes: geometry(1), int64(1)
memory usage: 304.8 KB


In [77]:
# Set index to comid
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.set_index.html
base_df_catch_geom.set_index('comid', inplace=True)
rest_df_catch_geom.set_index('comid', inplace=True)

In [78]:
base_df_catch_geom.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 19496 entries, 2743190 to 932040326
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   geom_catchment  19496 non-null  geometry
dtypes: geometry(1)
memory usage: 304.6 KB


### Replace `geom_catchment` Object with Geometry DType

In [79]:
# merge
base_df_catch['geom_catchment'] = base_df_catch_geom['geom_catchment']
rest_df_catch['geom_catchment'] = rest_df_catch_geom['geom_catchment']

In [80]:
base_df_catch.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 19496 entries, 1748535 to 932040370
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   tp_load             19496 non-null  float64 
 1   tn_load             19496 non-null  float64 
 2   tss_load            19496 non-null  float64 
 3   catchment_hectares  19496 non-null  float64 
 4   watershed_hectares  19496 non-null  float64 
 5   tp_loadrate_ws      19496 non-null  float64 
 6   tn_loadrate_ws      19496 non-null  float64 
 7   tss_loadrate_ws     19496 non-null  float64 
 8   maflowv             19496 non-null  float64 
 9   geom_catchment      19496 non-null  geometry
 10  cluster             17358 non-null  category
 11  sub_focusarea       186 non-null    Int64   
 12  nord                18870 non-null  Int64   
 13  nordstop            18844 non-null  Int64   
 14  huc12               19496 non-null  category
 15  streamorder       

In [81]:
base_df_catch.head(3)

,tp_load,tn_load,tss_load,catchment_hectares,watershed_hectares,tp_loadrate_ws,tn_loadrate_ws,tss_loadrate_ws,maflowv,geom_catchment,cluster,sub_focusarea,nord,nordstop,huc12,streamorder,headwater,phase,fa_name
comid,,,,,,,,,,,,,,,,,,,
1748535,881.4618,10322.4299,390433.1419,6496.7052,6501.69,0.1357,1.5874,60.0509,43.699,"MULTIPOLYGON (((535287.346 4678015.900, 535140...",drb,<NA>,74914,74914,020401020302,1,1,NaN,NaN
1748537,296.6355,3165.6081,88090.7401,1663.1712,1664.46,0.1784,1.9019,52.9245,11.189,"MULTIPOLYGON (((532639.560 4678753.867, 532610...",drb,<NA>,74913,74913,020401020302,1,1,NaN,NaN
1748539,350.9217,2816.4257,117212.5160,1639.4128,1640.70,0.2139,1.7164,71.4407,11.223,"MULTIPOLYGON (((525043.074 4672557.985, 525013...",drb,<NA>,74921,74921,020401020305,1,1,NaN,NaN


In [82]:
rest_df_catch.head(3)

,tp_load,tn_load,tss_load,catchment_hectares,watershed_hectares,tp_loadrate_ws,tn_loadrate_ws,tss_loadrate_ws,maflowv,geom_catchment,cluster,sub_focusarea,nord,nordstop,huc12,streamorder,headwater,phase,fa_name
comid,,,,,,,,,,,,,,,,,,,
1748535,881.4618,10322.4299,390433.1419,6496.7052,6501.69,0.1357,1.5874,60.0509,43.699,"MULTIPOLYGON (((535287.346 4678015.900, 535140...",drb,<NA>,74914,74914,020401020302,1,1,NaN,NaN
1748537,296.6355,3165.6081,88090.7401,1663.1712,1664.46,0.1784,1.9019,52.9245,11.189,"MULTIPOLYGON (((532639.560 4678753.867, 532610...",drb,<NA>,74913,74913,020401020302,1,1,NaN,NaN
1748539,350.9217,2816.4257,117212.5160,1639.4128,1640.70,0.2139,1.7164,71.4407,11.223,"MULTIPOLYGON (((525043.074 4672557.985, 525013...",drb,<NA>,74921,74921,020401020305,1,1,NaN,NaN


## Read Cluster Info and Geometries from Database

In [67]:
# read clusters
cluster_select = 'SELECT * FROM spatial.clustersv'
cluster_df = gpd.read_postgis(cluster_select, con)

In [68]:
cluster_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   gid         8 non-null      int64   
 1   objectid    8 non-null      int64   
 2   labels      8 non-null      object  
 3   label_id    8 non-null      int64   
 4   sum_acres   8 non-null      float64 
 5   acres       8 non-null      float64 
 6   ncontrb_a   8 non-null      float64 
 7   acres_ans   8 non-null      int64   
 8   name        8 non-null      object  
 9   shape_leng  8 non-null      float64 
 10  shape_area  8 non-null      float64 
 11  geom        8 non-null      geometry
dtypes: float64(5), geometry(1), int64(4), object(2)
memory usage: 896.0+ bytes


In [69]:
cluster_df

,gid,objectid,labels,label_id,sum_acres,acres,ncontrb_a,acres_ans,name,shape_leng,shape_area,geom
0,1,0,Poconos and Kittatinny,5,8.662812e+05,8.662812e+05,8.655927e+05,865593,Poconos and Kittatinny,5.179372,0.368441,"MULTIPOLYGON (((511922.413 4619793.062, 511948..."
1,2,0,Middle Schuylkill,3,5.018739e+05,5.018739e+05,5.015325e+05,501533,Middle Schuylkill,0.000000,0.000000,"MULTIPOLYGON (((398250.125 4483059.000, 398241..."
2,3,0,Brandywine and Christina,1,3.605241e+05,3.605241e+05,3.602701e+05,360270,Brandywine and Christina,0.000000,0.000000,"MULTIPOLYGON (((448782.249 4434220.000, 448840..."
3,4,0,New Jersey Highlands,4,4.417943e+05,4.417943e+05,4.414416e+05,441442,New Jersey Highlands,0.000000,0.000000,"MULTIPOLYGON (((486254.217 4533019.500, 486403..."
4,5,0,Schuylkill Highlands,6,1.091863e+05,1.091863e+05,1.091078e+05,109108,Schuylkill Highlands,0.000000,0.000000,"MULTIPOLYGON (((445764.281 4456450.500, 445828..."
5,6,0,Upstream Suburban Philadelphia,8,8.465670e+04,8.465670e+04,8.458919e+04,84589,Upstream Suburban Philadelphia,0.000000,0.000000,"MULTIPOLYGON (((479126.093 4455055.500, 479194..."
6,7,0,Kirkwood - Cohansey Aquifer,2,1.355948e+06,1.355948e+06,1.354892e+06,1354892,Kirkwood Cohansey Aquifer,0.000000,0.000000,"MULTIPOLYGON (((463626.226 4378209.091, 463595..."
7,8,0,Upper Lehigh,7,4.894516e+05,4.894516e+05,4.890917e+05,489092,Upper Lehigh,0.000000,0.000000,"MULTIPOLYGON (((465523.780 4570593.500, 465573..."


In [70]:
# read focus areas 
fa_select_P2P = 'select * from datadrwi.focusareas_p2plus where phase like \'Phase 2 Plus\' order by cluster, name_final, name'
fa_df_P2P = gpd.read_postgis(fa_select_P2P , con)


fa_select_P2 = 'select * from datadrwi.focusareas_p2plus where phase like \'Phase 2\' order by cluster, name_final, name'
fa_df_P2 = gpd.read_postgis(fa_select_P2 , con)

### Set DTypes and Index

In [71]:
cluster_df.labels = cluster_df.labels.astype('category')
cluster_df.name   = cluster_df.name.astype('category')

In [72]:
# Confirm Cluster name spelling in base model outputs to select which column to use as index
base_df.cluster.unique()

['drb', 'New Jersey Highlands', 'Poconos and Kittatinny', NaN, 'Upper Lehigh', 'Kirkwood - Cohansey Aquifer', 'Upstream Suburban Philadelphia', 'Brandywine and Christina', 'Middle Schuylkill', 'Schuylkill Highlands']
Categories (9, object): ['Brandywine and Christina', 'Kirkwood - Cohansey Aquifer', 'Middle Schuylkill', 'New Jersey Highlands', ..., 'Schuylkill Highlands', 'Upper Lehigh', 'Upstream Suburban Philadelphia', 'drb']

In [73]:
cluster_df.set_index('labels', inplace=True)

In [74]:
cluster_df

,gid,objectid,label_id,sum_acres,acres,ncontrb_a,acres_ans,name,shape_leng,shape_area,geom
labels,,,,,,,,,,,
Poconos and Kittatinny,1,0,5,8.662812e+05,8.662812e+05,8.655927e+05,865593,Poconos and Kittatinny,5.179372,0.368441,"MULTIPOLYGON (((511922.413 4619793.062, 511948..."
Middle Schuylkill,2,0,3,5.018739e+05,5.018739e+05,5.015325e+05,501533,Middle Schuylkill,0.000000,0.000000,"MULTIPOLYGON (((398250.125 4483059.000, 398241..."
Brandywine and Christina,3,0,1,3.605241e+05,3.605241e+05,3.602701e+05,360270,Brandywine and Christina,0.000000,0.000000,"MULTIPOLYGON (((448782.249 4434220.000, 448840..."
New Jersey Highlands,4,0,4,4.417943e+05,4.417943e+05,4.414416e+05,441442,New Jersey Highlands,0.000000,0.000000,"MULTIPOLYGON (((486254.217 4533019.500, 486403..."
Schuylkill Highlands,5,0,6,1.091863e+05,1.091863e+05,1.091078e+05,109108,Schuylkill Highlands,0.000000,0.000000,"MULTIPOLYGON (((445764.281 4456450.500, 445828..."
Upstream Suburban Philadelphia,6,0,8,8.465670e+04,8.465670e+04,8.458919e+04,84589,Upstream Suburban Philadelphia,0.000000,0.000000,"MULTIPOLYGON (((479126.093 4455055.500, 479194..."
Kirkwood - Cohansey Aquifer,7,0,2,1.355948e+06,1.355948e+06,1.354892e+06,1354892,Kirkwood Cohansey Aquifer,0.000000,0.000000,"MULTIPOLYGON (((463626.226 4378209.091, 463595..."
Upper Lehigh,8,0,7,4.894516e+05,4.894516e+05,4.890917e+05,489092,Upper Lehigh,0.000000,0.000000,"MULTIPOLYGON (((465523.780 4570593.500, 465573..."


In [75]:
cluster_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
CategoricalIndex: 8 entries, Poconos and Kittatinny to Upper Lehigh
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   gid         8 non-null      int64   
 1   objectid    8 non-null      int64   
 2   label_id    8 non-null      int64   
 3   sum_acres   8 non-null      float64 
 4   acres       8 non-null      float64 
 5   ncontrb_a   8 non-null      float64 
 6   acres_ans   8 non-null      int64   
 7   name        8 non-null      category
 8   shape_leng  8 non-null      float64 
 9   shape_area  8 non-null      float64 
 10  geom        8 non-null      geometry
dtypes: category(1), float64(5), geometry(1), int64(4)
memory usage: 1.4 KB


## Get the Protection Projects "Avoided Loads"

Returns the results for the protection projects uploaded into FieldDoc. Protection projects were exploaded from multi-polygon to polygon then intersected with NHDplus COMID.

The avoided load is taken as the Developed load - the Forested load. Assumes 100% developement to Medium Intensity (NLCD class 23). Only the average forested loading rate is provided in this table (averagess NLCD codes 41,42,43) in order to make the table simpler, however each land cover class' loading rate is explicitly used in the "avoided" load calculation.

Primary Key
- practice_i = FieldDoc (FD) primary key ID
- rn = Row number for number of polygons in the multipolygon submitted to FD as a project
- comid = NHDplus COMID intesecting with the protection polygon

SQL:
```sql
CONSTRAINT pk_protection_lbsavoided_fd PRIMARY KEY (practice_i, rn, comid)
```

**TO DO** in the future:
- Implement a Pandas/GeoPandas multi-index:  
https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html

In [8]:
%%time

db_connection_url = "postgresql://{}:{}@{}:{}/{}".format(_user[0], _password[0], _host[0], _port, _database[0])
con = create_engine(db_connection_url)  

tablename_prot = 'protection_lbsavoided_fd'

prot_select = 'SELECT * FROM datapolassess.{}'.format(tablename_prot)

prot_proj_df = gpd.read_postgis(prot_select, con)

Wall time: 1.08 s


In [9]:
prot_proj_df.head()

,practice_i,practice_n,rn,comid,practice_t,practice_d,project_na,project_st,creator_na,program_id,...,parcel_devtssload_lbyr,parcel_foretssload_lbyr,parcel_tssload_lbyr_avoided,parcel_devtnload_lbyr,parcel_foretnload_lbyr,parcel_tnload_lbyr_avoided,parcel_devtpload_lbyr,parcel_foretpload_lbyr,parcel_tpload_lbyr_avoided,geom
0,5289.0,Bear Creek,1,4185445,Fee acquisition,Acquisition,Bear Creek - Crystal Lake,complete,Dawn Gorham,5.0,...,175779.645,9187.118,166592.527,205.600,17.768,187.832,76.994,4.230,72.764,"MULTIPOLYGON (((-75.79484 41.17775, -75.79468 ..."
1,5289.0,Bear Creek,1,4185461,Fee acquisition,Acquisition,Bear Creek - Crystal Lake,complete,Dawn Gorham,5.0,...,159240.637,8322.707,150917.930,186.255,16.096,170.159,69.750,3.832,65.918,"MULTIPOLYGON (((-75.78786 41.18276, -75.78756 ..."
2,5289.0,Bear Creek,1,4185483,Fee acquisition,Acquisition,Bear Creek - Crystal Lake,complete,Dawn Gorham,5.0,...,235688.406,12318.247,223370.159,275.672,23.823,251.849,103.235,5.672,97.563,"MULTIPOLYGON (((-75.81161 41.17841, -75.81176 ..."
3,5289.0,Bear Creek,1,4185485,Fee acquisition,Acquisition,Bear Creek - Crystal Lake,complete,Dawn Gorham,5.0,...,48109.916,2514.463,45595.453,56.272,4.863,51.409,21.073,1.158,19.915,"MULTIPOLYGON (((-75.80381 41.17354, -75.80405 ..."
4,5289.0,Bear Creek,1,4185505,Fee acquisition,Acquisition,Bear Creek - Crystal Lake,complete,Dawn Gorham,5.0,...,40026.507,2083.105,37943.402,34.930,3.776,31.154,16.804,0.944,15.860,"MULTIPOLYGON (((-75.79805 41.17363, -75.80381 ..."


In [10]:
prot_proj_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 354 entries, 0 to 353
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype   
---  ------                        --------------  -----   
 0   practice_i                    354 non-null    float64 
 1   practice_n                    354 non-null    object  
 2   rn                            354 non-null    int64   
 3   comid                         354 non-null    int64   
 4   practice_t                    354 non-null    object  
 5   practice_d                    74 non-null     object  
 6   project_na                    354 non-null    object  
 7   project_st                    354 non-null    object  
 8   creator_na                    354 non-null    object  
 9   program_id                    354 non-null    float64 
 10  program_na                    354 non-null    object  
 11  created                       354 non-null    object  
 12  modified                      354 non-null

### Convert DataTypes

- Some `float64` columns should be converted to the nullable integers (`pd.Int64Dtype()`).
- Many string `object` columns can be converted to categoricals.
- Date and time string `object` columns should be parsed to DateTimes (`datetime64`).

Both of these improve performance, memory use, and compression. Floats are generally problematic for any index or field used as an index.

Docs:
- https://anaconda.org/TomAugspurger/pandas-performance/notebook
- https://pandas.pydata.org/pandas-docs/stable/user_guide/integer_na.html
- https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html
- https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html

In [11]:
# Convert IDs to nullable integers
prot_proj_df.practice_i = prot_proj_df.practice_i.astype(pd.Int64Dtype())
prot_proj_df.program_id = prot_proj_df.program_id.astype(pd.Int64Dtype())

In [12]:
# Convert string Objects to Categoricals
prot_proj_df.practice_n = prot_proj_df.practice_n.astype('category')
prot_proj_df.practice_t = prot_proj_df.practice_t.astype('category')
prot_proj_df.practice_d = prot_proj_df.practice_d.astype('category')
prot_proj_df.project_na = prot_proj_df.project_na.astype('category')
prot_proj_df.project_st = prot_proj_df.project_st.astype('category')
prot_proj_df.creator_na = prot_proj_df.creator_na.astype('category')
prot_proj_df.program_na = prot_proj_df.program_na.astype('category')
prot_proj_df.huc12      = prot_proj_df.huc12.astype('category')

In [13]:
# Convert date string Objects to DateTime
prot_proj_df.created  = pd.to_datetime(prot_proj_df.created, infer_datetime_format=True)
prot_proj_df.modified = pd.to_datetime(prot_proj_df.modified, infer_datetime_format=True)

In [14]:
prot_proj_df.iloc[:, 8:17].head()

,creator_na,program_id,program_na,created,modified,practice_url,project_url,huc12,area_acres
0,Dawn Gorham,5,Delaware River Watershed Protection Fund - For...,2019-01-04 19:03:54.420311,2019-11-15 20:56:34.050925,https://www.fielddoc.org/practices/5289,https://www.fielddoc.org/projects/2665,020401060203,84.609
1,Dawn Gorham,5,Delaware River Watershed Protection Fund - For...,2019-01-04 19:03:54.420311,2019-11-15 20:56:34.050925,https://www.fielddoc.org/practices/5289,https://www.fielddoc.org/projects/2665,020401060203,76.648
2,Dawn Gorham,5,Delaware River Watershed Protection Fund - For...,2019-01-04 19:03:54.420311,2019-11-15 20:56:34.050925,https://www.fielddoc.org/practices/5289,https://www.fielddoc.org/projects/2665,020401060203,113.445
3,Dawn Gorham,5,Delaware River Watershed Protection Fund - For...,2019-01-04 19:03:54.420311,2019-11-15 20:56:34.050925,https://www.fielddoc.org/practices/5289,https://www.fielddoc.org/projects/2665,020401060203,23.157
4,Dawn Gorham,5,Delaware River Watershed Protection Fund - For...,2019-01-04 19:03:54.420311,2019-11-15 20:56:34.050925,https://www.fielddoc.org/practices/5289,https://www.fielddoc.org/projects/2665,020401060204,18.881


In [15]:
prot_proj_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 354 entries, 0 to 353
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   practice_i                    354 non-null    Int64         
 1   practice_n                    354 non-null    category      
 2   rn                            354 non-null    int64         
 3   comid                         354 non-null    int64         
 4   practice_t                    354 non-null    category      
 5   practice_d                    74 non-null     category      
 6   project_na                    354 non-null    category      
 7   project_st                    354 non-null    category      
 8   creator_na                    354 non-null    category      
 9   program_id                    354 non-null    Int64         
 10  program_na                    354 non-null    category      
 11  created                 

## Get the Restoration Projects intersected by COMID

The FieldDoc primary key is `practice_i`, but we subdivide by `comid`, creating a multi-index.

SQL for the Primary Key:
```sql
CONSTRAINT pk_cache_restoration_lbsreduced PRIMARY KEY (comid, practice_i)
```

**TO DO** in the future:
- Implement a Pandas/GeoPandas multi-index:  
https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html

In [102]:
%%time
tablename_rest_projects = 'restoration_lbsreduced'
rest_proj_select = 'SELECT * FROM datapolassess.{}'.format(tablename_rest_projects)
rest_proj_df = gpd.read_postgis(rest_proj_select, con)

CPU times: user 86.2 ms, sys: 18.8 ms, total: 105 ms
Wall time: 4.21 s


In [103]:
rest_proj_df.head(3)

,comid,practice_id,site_id,practice_type,tn_reduced_lbs,tp_reduced_lbs,tss_reduced_lbs,practice_name,practice_description,project_name,project_status,creator_name,program_id,program_name,created_on,modified_on,practice_url,project_url,geom
0,2583213,5396,2864,Forest Buffer,2.8553,0.6454,331.0068,Forest buffer,None,EZG #53363 Restoring Paulins Kill Floodplain F...,complete,Kristine Rogers,1,Delaware River Restoration Fund,2019-01-24,2021-07-23,https://www.fielddoc.org/practices/5396,https://www.fielddoc.org/projects/2737,"MULTIPOLYGON (((522215.487 4552852.203, 522202..."
1,2583213,5399,2865,Forest Buffer,14.1300,3.4400,1438.7700,Forest buffer,None,EZG #53363 Restoring Paulins Kill Floodplain F...,complete,Kristine Rogers,1,Delaware River Restoration Fund,2019-01-24,2021-08-10,https://www.fielddoc.org/practices/5399,https://www.fielddoc.org/projects/2737,"MULTIPOLYGON (((522234.173 4553442.738, 522208..."
2,2583231,5396,2864,Forest Buffer,33.6847,7.6146,3904.9832,Forest buffer,None,EZG #53363 Restoring Paulins Kill Floodplain F...,complete,Kristine Rogers,1,Delaware River Restoration Fund,2019-01-24,2021-07-23,https://www.fielddoc.org/practices/5396,https://www.fielddoc.org/projects/2737,"MULTIPOLYGON (((522215.487 4552852.203, 522202..."


In [104]:
rest_proj_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   comid                 918 non-null    int64   
 1   practice_id           918 non-null    int64   
 2   site_id               918 non-null    int64   
 3   practice_type         918 non-null    object  
 4   tn_reduced_lbs        918 non-null    float64 
 5   tp_reduced_lbs        918 non-null    float64 
 6   tss_reduced_lbs       918 non-null    float64 
 7   practice_name         918 non-null    object  
 8   practice_description  564 non-null    object  
 9   project_name          918 non-null    object  
 10  project_status        918 non-null    object  
 11  creator_name          918 non-null    object  
 12  program_id            918 non-null    int64   
 13  program_name          918 non-null    object  
 14  created_on            918 non-null    object  
 15

### Convert DataTypes

As for Project Projects, with rational and methods described above.

In [105]:
# Convert IDs to nullable integers
rest_proj_df.practice_id = rest_proj_df.practice_id.astype(pd.Int64Dtype())
rest_proj_df.program_id = rest_proj_df.program_id.astype(pd.Int64Dtype())
rest_proj_df.site_id    = rest_proj_df.site_id.astype(pd.Int64Dtype())

In [106]:
# Convert string Objects to Categoricals
rest_proj_df.practice_type = rest_proj_df.practice_type.astype('category')
rest_proj_df.practice_name = rest_proj_df.practice_name.astype('category')
rest_proj_df.practice_description = rest_proj_df.practice_description.astype('category')

rest_proj_df.project_name = rest_proj_df.project_name.astype('category')
rest_proj_df.project_status = rest_proj_df.project_status.astype('category')
rest_proj_df.creator_name = rest_proj_df.creator_name.astype('category')

rest_proj_df.program_name = rest_proj_df.program_name.astype('category')

In [107]:
# Convert date string Objects to DateTime
rest_proj_df.created_on  = pd.to_datetime(rest_proj_df.created_on, infer_datetime_format=True)
rest_proj_df.modified_on = pd.to_datetime(rest_proj_df.modified_on, infer_datetime_format=True)

In [108]:
rest_proj_df.head(3)

,comid,practice_id,site_id,practice_type,tn_reduced_lbs,tp_reduced_lbs,tss_reduced_lbs,practice_name,practice_description,project_name,project_status,creator_name,program_id,program_name,created_on,modified_on,practice_url,project_url,geom
0,2583213,5396,2864,Forest Buffer,2.8553,0.6454,331.0068,Forest buffer,NaN,EZG #53363 Restoring Paulins Kill Floodplain F...,complete,Kristine Rogers,1,Delaware River Restoration Fund,2019-01-24,2021-07-23,https://www.fielddoc.org/practices/5396,https://www.fielddoc.org/projects/2737,"MULTIPOLYGON (((522215.487 4552852.203, 522202..."
1,2583213,5399,2865,Forest Buffer,14.1300,3.4400,1438.7700,Forest buffer,NaN,EZG #53363 Restoring Paulins Kill Floodplain F...,complete,Kristine Rogers,1,Delaware River Restoration Fund,2019-01-24,2021-08-10,https://www.fielddoc.org/practices/5399,https://www.fielddoc.org/projects/2737,"MULTIPOLYGON (((522234.173 4553442.738, 522208..."
2,2583231,5396,2864,Forest Buffer,33.6847,7.6146,3904.9832,Forest buffer,NaN,EZG #53363 Restoring Paulins Kill Floodplain F...,complete,Kristine Rogers,1,Delaware River Restoration Fund,2019-01-24,2021-07-23,https://www.fielddoc.org/practices/5396,https://www.fielddoc.org/projects/2737,"MULTIPOLYGON (((522215.487 4552852.203, 522202..."


In [109]:
rest_proj_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   comid                 918 non-null    int64         
 1   practice_id           918 non-null    Int64         
 2   site_id               918 non-null    Int64         
 3   practice_type         918 non-null    category      
 4   tn_reduced_lbs        918 non-null    float64       
 5   tp_reduced_lbs        918 non-null    float64       
 6   tss_reduced_lbs       918 non-null    float64       
 7   practice_name         918 non-null    category      
 8   practice_description  564 non-null    category      
 9   project_name          918 non-null    category      
 10  project_status        918 non-null    category      
 11  creator_name          918 non-null    category      
 12  program_id            918 non-null    Int64         
 13  program_name

## Get the Point Sources by NPDES ID and COMID

In [110]:
%%time
tablename_pointsource = 'ms_pointsource_drb_12_13_18'
pointsource_select = 'SELECT * FROM wikiwtershed.{}'.format(tablename_pointsource)
point_source_df = gpd.read_postgis(pointsource_select, con)

CPU times: user 49.7 ms, sys: 4.08 ms, total: 53.8 ms
Wall time: 818 ms


In [111]:
point_source_df.head()

,ogc_fid,geom,npdes_id,city,state,latitude,longitude,huc12,avg_n_conc,lbsn_yr,mgd,avgpconc,lbsp_yr,kgn_yr,kgp_yr,facilityname,comid
0,1,POINT (414070.808 4469871.626),PA0033995,BERN TWP,PA,40.375000,-76.012222,020402030409,0.191,84.591269,0.163050,0.191,1325.042759,38.369923,601.028795,COUNTY OF BERKS WWTP,4783187
1,1,POINT (450085.743 4495200.414),PA0051811,SOUTH WHITEHALL1TWP,PA,40.606111,-75.590000,020401060703,35.000,32.011082,0.000300,35.000,2.733729,14.519971,1.239998,LEHIGH COUNTY AUTH,4187751
2,1,POINT (443086.168 4439655.703),1594403,WEST VINCENT TWP,PA,40.105277,-75.667777,020402031003,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,MATTHEWS MEADOWS STP,4781791
3,1,POINT (444600.842 4439582.877),1592417,WEST VINCENT TWP,PA,40.104722,-75.650000,020402031003,0.000,0.000000,0.000000,0.000,NaN,0.000000,NaN,SAINT STEPHEN'S GREENE STP,4782621
4,1,POINT (499574.949 4511288.150),NJ0065196,Township of Washington,NJ,40.752548,-75.005035,020401050401,7.175,155.557987,0.000677,7.175,5.599735,70.559859,2.539995,390 RT 57,2588253


In [112]:
point_source_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 812 entries, 0 to 811
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   ogc_fid       812 non-null    int64   
 1   geom          812 non-null    geometry
 2   npdes_id      812 non-null    object  
 3   city          789 non-null    object  
 4   state         812 non-null    object  
 5   latitude      812 non-null    float64 
 6   longitude     812 non-null    float64 
 7   huc12         812 non-null    object  
 8   avg_n_conc    811 non-null    float64 
 9   lbsn_yr       811 non-null    float64 
 10  mgd           812 non-null    float64 
 11  avgpconc      811 non-null    float64 
 12  lbsp_yr       809 non-null    float64 
 13  kgn_yr        811 non-null    float64 
 14  kgp_yr        809 non-null    float64 
 15  facilityname  812 non-null    object  
 16  comid         812 non-null    int64   
dtypes: float64(9), geometry(1), int64(2), object(5

### Convert DataTypes

As for Project Projects, with rational and methods described above.

In [113]:
# Convert string Objects to Categoricals
point_source_df.npdes_id = point_source_df.npdes_id.astype('category')
point_source_df.city     = point_source_df.city.astype('category')
point_source_df.state    = point_source_df.state.astype('category')
point_source_df.huc12 = point_source_df.huc12.astype('category')
point_source_df.facilityname = point_source_df.facilityname.astype('category')

In [114]:
point_source_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 812 entries, 0 to 811
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   ogc_fid       812 non-null    int64   
 1   geom          812 non-null    geometry
 2   npdes_id      812 non-null    category
 3   city          789 non-null    category
 4   state         812 non-null    category
 5   latitude      812 non-null    float64 
 6   longitude     812 non-null    float64 
 7   huc12         812 non-null    category
 8   avg_n_conc    811 non-null    float64 
 9   lbsn_yr       811 non-null    float64 
 10  mgd           812 non-null    float64 
 11  avgpconc      811 non-null    float64 
 12  lbsp_yr       809 non-null    float64 
 13  kgn_yr        811 non-null    float64 
 14  kgp_yr        809 non-null    float64 
 15  facilityname  812 non-null    category
 16  comid         812 non-null    int64   
dtypes: category(5), float64(9), geometry(1), int64

### Set Index to `npdes_id`

`npdes_id` appears to be the Primary Key

In [115]:
# Confirm that the `npdes_id` hase one unique value per data row.
point_source_df.npdes_id.unique()

['PA0033995', 'PA0051811', '1594403', '1592417', 'NJ0065196', ..., 'NJ0022985', 'PA0056081', 'PA0036081', 'PA0026638', 'NJ0104388']
Length: 812
Categories (812, object): ['1503410', '1505402', '1505412', '1508411', ..., 'PAG053550', 'PAG053603', 'PAG113500', 'WQG02231503']

In [116]:
# Confirm that the `npdes_id` hase one unique value per data row, alternate approach.
# Outputs are generally sorted from highest to lowest counts
point_source_df.npdes_id.value_counts()

1503410        1
PA0050016      1
PA0043982      1
PA0044024      1
PA0044423      1
              ..
NJ0169030      1
NJ0170330      1
NJ0171565      1
NJ0171620      1
WQG02231503    1
Name: npdes_id, Length: 812, dtype: int64

In [117]:
# Set index to 'npdes_id'
point_source_df.set_index('npdes_id', inplace=True)

In [118]:
point_source_df.index

CategoricalIndex(['PA0033995', 'PA0051811', '1594403', '1592417', 'NJ0065196',
                  'PA0012238', 'PA0026867', 'PA0030023', 'PA0011185',
                  'NJ0004278',
                  ...
                  'PA0028975', 'NJG0175200', 'NJ0022250', 'NJ0024040',
                  'PA0055671', 'NJ0022985', 'PA0056081', 'PA0036081',
                  'PA0026638', 'NJ0104388'],
                 categories=['1503410', '1505402', '1505412', '1508411', '1586408', '1586409', '1587447', '1590416', ...], ordered=False, dtype='category', name='npdes_id', length=812)

In [119]:
point_source_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
CategoricalIndex: 812 entries, PA0033995 to NJ0104388
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   ogc_fid       812 non-null    int64   
 1   geom          812 non-null    geometry
 2   city          789 non-null    category
 3   state         812 non-null    category
 4   latitude      812 non-null    float64 
 5   longitude     812 non-null    float64 
 6   huc12         812 non-null    category
 7   avg_n_conc    811 non-null    float64 
 8   lbsn_yr       811 non-null    float64 
 9   mgd           812 non-null    float64 
 10  avgpconc      811 non-null    float64 
 11  lbsp_yr       809 non-null    float64 
 12  kgn_yr        811 non-null    float64 
 13  kgp_yr        809 non-null    float64 
 14  facilityname  812 non-null    category
 15  comid         812 non-null    int64   
dtypes: category(4), float64(9), geometry(1), int64(2)
memory usage: 165.5 

## Get Model My Watershed output Loads & Loading Rates for all HUC12s 

In [120]:
%%time
# Model My Watershed outputs for all HUC12s
mmw_huc12_loads_df = pd.read_csv(Path.cwd() / 'mmw_huc12_loads' / 'mmw_drbloads.csv',
                                dtype={'huc12': 'category'},
                                index_col='huc12',
                               )
# Iterate this block with output form below

CPU times: user 9.26 ms, sys: 2.45 ms, total: 11.7 ms
Wall time: 10.6 ms


In [121]:
mmw_huc12_loads_df.info()

<class 'pandas.core.frame.DataFrame'>
CategoricalIndex: 484 entries, 020402031005 to 020403020204
Data columns (total 77 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   gid                     484 non-null    int64  
 1   area_sqkm               484 non-null    float64
 2   tss_hay_pas_kg          484 non-null    float64
 3   tss_crop_kg             484 non-null    float64
 4   tss_wooded_kg           484 non-null    float64
 5   tss_wetland_kg          484 non-null    float64
 6   tss_open_land_kg        484 non-null    float64
 7   tss_barren_kg           484 non-null    float64
 8   tss_dev_ld_kg           484 non-null    float64
 9   tss_dev_md_kg           484 non-null    float64
 10  tss_dev_hd_kg           484 non-null    float64
 11  tss_dev_os_kg           484 non-null    float64
 12  tss_farm_animal_kg      484 non-null    float64
 13  tss_streambank_eros_kg  484 non-null    float64
 14  tss_subsurface_k

In [122]:
mmw_huc12_loads_df

,gid,area_sqkm,tss_hay_pas_kg,tss_crop_kg,tss_wooded_kg,tss_wetland_kg,tss_open_land_kg,tss_barren_kg,tss_dev_ld_kg,tss_dev_md_kg,...,tp_hay_pas_lb_acre,tp_crop_lb_acre,tp_wooded_lb_acre,tp_wetland_lb_acre,tp_open_land_lb_acre,tp_barren_lb_acre,tp_dev_ld_lb_acre,tp_dev_md_lb_acre,tp_dev_hd_lb_acre,tp_dev_os_lb_acre
huc12,,,,,,,,,,,,,,,,,,,,,
020402031005,325,54.86,72381.6,752464.8,2435.8,164.0,397.2,4.8,17031.1,38503.9,...,0.442212,2.296956,0.008400,0.025203,0.072890,0.013396,0.034463,0.134420,0.148949,0.034173
020401060204,181,142.92,5095.9,5405.5,8655.6,586.0,860.1,18.5,1449.1,3503.5,...,0.532758,0.794506,0.006892,0.012503,0.035351,0.016409,0.029567,0.110337,0.177020,0.027924
020403020107,465,52.01,840.5,59579.0,2096.4,647.3,185.3,6.4,1059.3,3650.2,...,0.066846,0.237004,0.001139,0.015130,0.014578,0.013396,0.026087,0.112895,0.218671,0.024071
020402030406,290,80.52,99470.0,1620216.1,7201.4,178.8,135.6,0.0,4970.5,9045.3,...,0.166861,1.315266,0.004691,0.020427,0.060079,0.000000,0.033497,0.138338,0.150254,0.031000
020402070401,410,53.98,33580.1,903158.1,898.6,1231.8,1031.4,0.0,1062.1,2381.4,...,0.082273,0.361225,0.002546,0.012767,0.026025,0.000000,0.029156,0.127158,0.153417,0.027978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
020401040601,109,44.05,923.5,0.0,4124.5,25.6,817.9,0.0,0.0,0.0,...,0.475829,0.000000,0.008857,0.015766,0.053378,0.000000,0.000000,0.000000,0.000000,0.036715
020401010102,2,65.08,122689.7,127380.0,10611.7,130.3,0.0,0.0,140.4,25.2,...,0.388249,0.622149,0.007713,0.014734,0.000000,0.000000,0.045753,0.000000,0.000000,0.024191
020402030608,304,57.26,32389.6,715473.5,21282.0,64.7,2686.0,117.7,1386.3,2439.9,...,0.216267,2.575598,0.007773,0.017290,0.106007,0.021708,0.030922,0.136528,0.137682,0.027023


# Save to Parquet Files
The code below converts the data into a locally saved parquet file to avoid having to access the database every time we run the visualization script.  

Apache Parquet has become the high-performance binary cloud format of choice for storing dataframes.
- https://pandas.pydata.org/docs/user_guide/io.html#io-parquet
- https://anaconda.org/TomAugspurger/pandas-performance/notebook
- https://geopandas.readthedocs.io/en/latest/docs/reference/api/geopandas.GeoDataFrame.to_parquet.html

NOTE: The current version of GeoPandas only supports the `pyarrow` engine. In future implementations, look for integration with `fastparquet` engine/library, which is implemented in Pandas and more tightly integrated with numba and dask. See https://fastparquet.readthedocs.io

In [83]:
# Parquet requires pyarrow
import pyarrow

Although Parquet has been tightly integrated with Pandas for a long time and is very stable, the GeoPandas Parquet implementation is relatively new and comes with this warning:

```
# UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
```

In [84]:
import warnings
warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')

In [85]:
# Find current working directory
from pathlib import Path
Path.cwd()

WindowsPath('C:/Users/sjordan/Documents/GitHub/WikiSRATMicroService')

In [86]:
# Set relative path - will work for anybody in this directory / cloning the github
data_folder    = Path('data/')

In [87]:
%%time

# Save Model Output Data
# Use the `gzip` compression engine, for bettter (~2/3) but slower (~5x write) compression
# than the default `snappy` engine.

base_df_reach.to_parquet(data_folder /'base_df_reach.parquet',compression='gzip')
rest_df_reach.to_parquet(data_folder /'rest_df_reach.parquet',compression='gzip')

base_df_catch.to_parquet(data_folder /'base_df_catch.parquet',compression='gzip')
rest_df_catch.to_parquet(data_folder /'rest_df_catch.parquet',compression='gzip')

Wall time: 5.91 s


In [128]:
cluster_df.to_parquet(data_folder / 'cluster_df.parquet', compression='gzip')

In [19]:
%%time
# Save Protection Project Data
prot_proj_df.to_parquet(data_folder /'prot_proj_df.parquet',compression='gzip')

Wall time: 146 ms


In [130]:
%%time
# Save Restoration Project Data
rest_proj_df.to_parquet(data_folder /'rest_proj_df.parquet',compression='gzip')

CPU times: user 77.4 ms, sys: 3.24 ms, total: 80.6 ms
Wall time: 79.8 ms


In [131]:
%%time
# Save Point Source Data
point_source_df.to_parquet(data_folder /'point_source_df.parquet',compression='gzip')

CPU times: user 38.1 ms, sys: 2.31 ms, total: 40.5 ms
Wall time: 40 ms


In [132]:
%%time
# Save MMW HUC12 Loads
mmw_huc12_loads_df.to_parquet(data_folder /'mmw_huc12_loads_df.parquet',compression='gzip')

CPU times: user 29.8 ms, sys: 2.58 ms, total: 32.4 ms
Wall time: 33.7 ms


In [20]:
fa_df_P2P.to_parquet(data_folder /'fa_phase2plus_df.parquet', compression = 'gzip')
fa_df_P2.to_parquet(data_folder /'fa_phase2_df.parquet', compression = 'gzip')